In [60]:
# !pip install tensorflow

In [61]:
import os # операционная система (работа с папкой)
import requests # запросы по url
import re # регулярные выражения
from bs4 import BeautifulSoup # разобрать html
import io # работа с файлами (загрузка / выгрузка)

# from google.colab import files # скачать из colab

# Классификация нового изображения
from PIL import Image
from torchvision import transforms
import torch

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
import os

In [62]:
# Загрузка данных
GENERAL_URL = 'https://yandex.ru/images/search?text={query_text}&p={page}'
DIR_NAME_class01 = 'пиджак'
DIR_NAME_class02 = 'брюки'
ENCODING = 'utf-8'
MOZILLA_SEARCH_HEADER = {'User-Agent':'Mozilla/5.0'} # Браузер Мозилла
HTML_MODE = 'html.parser'
HREF_MODE = True # включить запись ссылок

In [63]:
# Функция парсера URL
def parser_url(url):
  pattern = r'img_url=([^&]+)&text='
  match_u = re.search(pattern, url)
  if match_u:
    img_url_encoded = match_u.group(1)
    img_url_decoded = img_url_encoded.replace('%2F', '/').replace('%3A', ':')
    print(img_url_decoded)
    return img_url_decoded
  else:
    print('Ссылка после img_url не найдена в URL')

In [64]:
# Функция загрузки 1 изображения
def download_image(url, save_path):
  try:
    response = requests.get(url, headers=MOZILLA_SEARCH_HEADER, stream=True)
    if(response.status_code == 200):
      with open(save_path, 'wb') as file:
        for chunk in response.iter_content(1024):
          file.write(chunk)
      return True
    else:
      print(f'Не удалось загрузить изображение: {url}')
      return False
  except Exception as e:
    print(f'Ошибка при загрузке изображения: {url}')
    return False

In [65]:
def get_links_on_keyword(keyword, page_num=1, encoding=ENCODING):
  '''
  Взять по ключевому слову набор ссылок с сайта ХабраХабр
  @param keyword - ключевое слово
  @param page_num - сколько страниц смотрим (как правило, на 1 странице 20 статей)
  '''
  links_to_find = []
  names_to_find = []
  __PARSE_TAG = 'a'
  __PARSE_CLASS = 'serp-item__link'

  total_len = page_num
  keyword_lwr = keyword.lower()
  downloaded_count = 0

  for page in range(page_num + 1):
    page_articles = GENERAL_URL.format(page=page_num, query_text=keyword)
    response = requests.get(page_articles, headers=MOZILLA_SEARCH_HEADER) # Браузер Мозилла
    response.encoding = encoding
    page_data_soup = BeautifulSoup(response.text, HTML_MODE)
    if(response.status_code == 200):
      for link in page_data_soup.find_all(__PARSE_TAG, HREF_MODE, class_=__PARSE_CLASS):
        print(link['href'])
        # получаем полный URL изображения
        img_url = parser_url(link['href'])
        print(img_url)
        links_to_find.append[img_url]
  return names_to_find, links_to_find

In [66]:
# Функция проверки директории
def check_repo_dataset(class_name, current_dir="./"):
  class_folder = os.path.join(current_dir + '/datasets', class_name)
  if not os.path.exists(class_folder):
    os.makedirs(class_folder)
    return class_folder
  else:
    return class_folder

In [67]:
# Функция колличество изображений в выдаче
def calc_pages(num_images):
  return num_images // 5 + (num_images % 5 > 0) if num_images > 5 else 1

In [68]:
# Функция загрузки изображений
def download_images(query, num_images, mini_images = False):
  pages = calc_pages(num_images)
  class_folder = check_repo_dataset(query, "./")
  downloaded_count = 0
  base_url = 'https:'
  # а вот это чтобы без движков было, грузим странички
  for page in range(0, pages):
    search_url = GENERAL_URL.format(query_text=query, page=page)
    response = requests.get(search_url, headers=MOZILLA_SEARCH_HEADER)
    soup = BeautifulSoup(response.text, 'html.parser')
    if (mini_images == False):
      for a in soup.find_all('a', class_='serp-item__link'):
        img_url = a['href']
        # получаем полный URL изображения
        img_url = parser_url(img_url)
        image_filename = f'{downloaded_count:04d}.jpg'
        image_path = os.path.join(class_folder, image_filename)
        if(download_image(img_url, image_path)):
          downloaded_count += 1
          print(f"Загружено изображений для {query}: {downloaded_count}/{num_images}")
        if(downloaded_count >= num_images):
          break
    else:
      for a in soup.find_all('img', class_='serp-item__thumb'):
        img_url = a['src']
        # из за получение //avatar, надо бы добавить https:// чтобы ссылка стала полной
        if(not img_url.startswith('http')):
          img_url = base_url + img_url
          image_filename = f'{downloaded_count:04d}.jpg'
          image_path = os.path.join(class_folder, image_filename)
          if(download_image(img_url, image_path)):
            downloaded_count += 1
            print(f'Загружено изображений для {query}: {downloaded_count}/{num_images}')
          if(downloaded_count >= num_images):
            break

In [69]:
# Загрузка 1 набора изображений
# download_images(DIR_NAME_class01, num_images=200, mini_images=True)

KeyboardInterrupt: 

In [71]:
# Загрузка 2 набора изображений
download_images(DIR_NAME_class02, num_images=200, mini_images=True)

Загружено изображений для брюки: 1/200
Загружено изображений для брюки: 2/200
Загружено изображений для брюки: 3/200
Загружено изображений для брюки: 4/200
Загружено изображений для брюки: 5/200
Загружено изображений для брюки: 6/200
Загружено изображений для брюки: 7/200
Загружено изображений для брюки: 8/200
Загружено изображений для брюки: 9/200
Загружено изображений для брюки: 10/200
Загружено изображений для брюки: 11/200
Загружено изображений для брюки: 12/200
Загружено изображений для брюки: 13/200
Загружено изображений для брюки: 14/200
Загружено изображений для брюки: 15/200
Загружено изображений для брюки: 16/200
Загружено изображений для брюки: 17/200
Загружено изображений для брюки: 18/200
Загружено изображений для брюки: 19/200
Загружено изображений для брюки: 20/200
Загружено изображений для брюки: 21/200
Загружено изображений для брюки: 22/200
Загружено изображений для брюки: 23/200
Загружено изображений для брюки: 24/200
Загружено изображений для брюки: 25/200
Загружено

In [75]:
# Архивация датасетов
# !zip -r '/content/{DIR_NAME_class01}.zip' '/content/datasets/{DIR_NAME_class01}'
# !zip -r '/content/{DIR_NAME_class02}.zip' '/content/datasets/{DIR_NAME_class02}'

# !zip -r './{DIR_NAME_class01}.zip' './datasets/{DIR_NAME_class01}'
# !zip -r './{DIR_NAME_class02}.zip' './datasets/{DIR_NAME_class02}'

"zip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
"zip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [ ]:
# Распаковка архива датасета
# !unzip -q '/content/{DIR_NAME_class01}.zip'
# !unzip -q '/content/{DIR_NAME_class02}.zip'

# !unzip -q './{DIR_NAME_class01}.zip'
# !unzip -q './{DIR_NAME_class02}.zip'

In [ ]:
# Получить файлы
folder = '/content/datasets/'
lst_files_01 = os.listdir(f'{folder}/chihuahua')[:50] # 1. Считываем из папки; 2. Обрезаем датасет, 1
lst_files_02 = os.listdir(f'{folder}/muffin')[:50] # 1. Считываем из папки; 2. Обрезаем датасет, 0

In [ ]:
!mkdir "train_data"
!mkdir "test_data"
!mkdir "validation_data"